In [1]:
%load_ext Cython

In [2]:
%timeit sum(range(100000))

1.29 ms ± 37.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
284 / 6.76

42.01183431952663

In [2]:
%%cython

from cython.parallel import prange, parallel

def mysum():
    cdef int i
    cdef int n = 10000
    with nogil, parallel(num_threads=8):
        for i in prange(n, schedule='static', chunksize=100):
            i + 1
print(mysum())

None


In [3]:
%%cython --cplus --compile-args=-stdlib=libc++ --link-args=-stdlib=libc++

def mysum2():
    cdef int i = 0
    cdef int n = 10000
    while i < n:
        i + 1
        i += 1
print(mysum2())

None


In [4]:
%timeit mysum()
%timeit mysum2()

91.6 ns ± 3.02 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
24.7 ns ± 0.0835 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [3]:
import psutil

In [4]:
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 2800.00Mhz
Min Frequency: 2800.00Mhz
Current Frequency: 2800.00Mhz
CPU Usage Per Core:
Core 0: 21.0%
Core 1: 0.0%
Core 2: 15.0%
Core 3: 2.0%
Core 4: 14.9%
Core 5: 0.0%
Core 6: 7.0%
Core 7: 0.0%
Total CPU Usage: 11.0%


In [5]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

In [6]:
# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

======================================== Memory Information ========================================
Total: 16.00GB
Available: 8.03GB
Used: 7.96GB
Percentage: 49.8%
==================== SWAP ====================
Total: 0.00B
Free: 0.00B
Used: 0.00B
Percentage: 0.0%
